In [ ]:
import boto3
import tenseal as ts
from deepface import DeepFace
import base64
import math
import deepface.commons.distance as dst
import random
import unittest

In [ ]:
aws_access_key = s3.access_key
aws_secret_key = s3.secret_key
s3 = boto3.client('s3', aws_access_key_id=aws_access_key, aws_secret_access_key=aws_secret_key)

In [ ]:
def write_data_to_s3(bucket, key, data):
    if type(data) == bytes:
        data = base64.b64encode(data)
    s3.put_object(Bucket=bucket, Key=key, Body=data)


In [ ]:
def read_data_from_s3(bucket, key):
    response = s3.get_object(Bucket=bucket, Key=key)
    data = response['Body'].read()
    return base64.b64decode(data)


In [ ]:
def euclidean_squared(v1, v2):
    return (v1 - v2).dot(v1 - v2)

In [ ]:
def main():
    img1_path = "../pics/andrew1.png"
    img2_path = "../pics/andrew4.png"
    img1_embedding = DeepFace.represent(img1_path, model_name='Facenet')[0]["embedding"]
    img2_embedding = DeepFace.represent(img2_path, model_name='Facenet')[0]["embedding"]

    context = ts.context(ts.SCHEME_TYPE.CKKS, poly_modulus_degree=8192, coeff_mod_bit_sizes=[60, 40, 40, 60])
    context.generate_galois_keys()
    context.global_scale = 2 ** 40
    secret_context = context.serialize(save_secret_key=True)
    write_data_to_s3("learnu7bucket", "secret.txt", secret_context)
    context.make_context_public()
    public_context = context.serialize()
    write_data_to_s3("learnu7bucket", "public.txt", public_context)
    del context, secret_context, public_context

    context = ts.context_from(read_data_from_s3("learnu7bucket", "secret.txt"))
    enc_v1 = ts.ckks_vector(context, img1_embedding)
    enc_v2 = ts.ckks_vector(context, img2_embedding)
    enc_v1_proto = enc_v1.serialize()
    enc_v2_proto = enc_v2.serialize()
    write_data_to_s3("learnu7bucket", "enc_v1.txt", enc_v1_proto)
    write_data_to_s3("learnu7bucket", "enc_v2.txt", enc_v2_proto)
    del context, enc_v1, enc_v2

    context = ts.context_from(read_data_from_s3("learnu7bucket", "public.txt"))
    enc_v1_proto = read_data_from_s3("learnu7bucket", "enc_v1.txt")
    enc_v1 = ts.lazy_ckks_vector_from(enc_v1_proto)
    enc_v1.link_context(context)
    enc_v2_proto = read_data_from_s3("learnu7bucket", "enc_v2.txt")
    enc_v2 = ts.lazy_ckks_vector_from(enc_v2_proto)
    enc_v2.link_context(context)

    euclidean_sq = euclidean_squared(enc_v1, enc_v2)
    write_data_to_s3("learnu7bucket", "euclidean_squared.txt", euclidean_sq.serialize())
    del context, enc_v1, enc_v2, euclidean_sq

    context = ts.context_from(read_data_from_s3("learnu7bucket", "secret.txt"))
    euclidean_sq_proto = read_data_from_s3("learnu7bucket", "euclidean_squared.txt")
    euclidean_sq = ts.lazy_ckks_vector_from(euclidean_sq_proto)
    euclidean_sq.link_context(context)
    euclidean_sq_plain = euclidean_sq.decrypt()[0]

    distance = dst.findEuclideanDistance(img1_embedding, img2_embedding)
    squared_distance = distance * distance
    if euclidean_sq_plain < 10:
        print("they are same person")
    else:
        print("they are different persons")

    print("euclidean squared - tradational: ", distance * distance)
    print("euclidean squared - homomorphic: ", euclidean_sq_plain)
    assert abs(distance * distance - euclidean_sq_plain) < 0.00001

if __name__ == "__main__":
    main()
